# **AutoGen Core - Distributed Runtime**

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

#### Start with our Message class

In [2]:
@dataclass
class Message:
    content: str

#### And now - a host for our distributed runtime

In [3]:

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

#### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

#### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [ ]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:   # All agents in different runtimes and interacting within different runtimes

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")

In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons in favor of choosing AutoGen for your AI Agent project:

1. **Modularity and Scalability**: AutoGen's framework is designed to be modular and extensible, allowing for the creation of scalable and customizable systems that can grow with your project needs.

2. **Ease of Use**: The integrated observability and debugging tools make it easier to monitor and control agent workflows, enhancing the development and maintenance process.

3. **Multi-Agent Collaboration**: AutoGen excels in facilitating multi-agent systems, allowing for complex interactions and collaborations between agents, which can lead to more efficient and powerful AI solutions.

4. **Customizable Agents**: The framework provides capabilities for creating customizable agents that can be easily tailored to specific requirements, ensuring flexibility in usage.

5. **Efficiency in Tasks**: By leveraging AutoGen’s tools, you can automate workflows and streamline processes, saving significant time and effort compared to traditional methods.

6. **Integration Capabilities**: AutoGen can integrate with various LLMs, tools, and human inputs, making it versatile for diverse applications.

These advantages make AutoGen a compelling choice for developing sophisticated AI agents in your project. 

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen in an AI Agent project:

1. **Less Structured**: AutoGen may offer less structure than other frameworks, which could complicate codebase management and maintainability, especially for larger projects.

2. **Accuracy Concerns**: There are challenges related to the accuracy of the models it employs, which could lead to unreliable outputs or behaviors in production settings.

3. **Complex Deployment and Maintenance**: Managing the deployment and ongoing maintenance of AI features in real-world scenarios can be challenging, potentially requiring more resources and expertise.

4. **Customization Complexity**: While it allows for customization, the need for deep reliance on frameworks can increase complexity, making it harder to implement tailored solutions.

5. **Team Collaboration Issues**: Although it supports multi-agent collaboration, the coordination and communication between agents can sometimes lead to inefficiencies if not properly handled.

Evaluating these drawbacks in the context of your specific project requirements is advisable before making a decision.

TERMINATE



## Decision:

Based on the research provided by your team, I recommend choosing AutoGen for your AI Agent project.

**Rationale**: The advantages of AutoGen, such as its modularity, scalability, ease of use, and multi-agent collaboration capabilities, significantly outweigh the concerns. While there are validity in the drawbacks regarding structure and accuracy, the benefits in efficiency and customization offer a strong case for leveraging AutoGen, especially if your project requires adaptability and collaboration among agents. With proper planning and resource allocation, the challenges can be managed effectively.

TERMINATE

In [10]:

await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [14]:
await host.stop()